In [1]:
import cv2
import numpy as np

def segment_objects(image_path, write_file=True):
    # load image
    image = cv2.imread(image_path)
    
    # convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # opencv internally uses BGR instead of RGB
    
    # define thresholds for background (white/grey)
    lower_bound = np.array([0, 0, 180]) 
    upper_bound = np.array([180, 50, 255])
    
    # create mask for background
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    
    # invert mask for object isolation
    mask_inv = cv2.bitwise_not(mask)
    
    # refine mask with morphologic operations
    kernel = np.ones((3,3),np.uint8)
    mask_inv = cv2.morphologyEx(mask_inv, cv2.MORPH_CLOSE, kernel)
    mask_inv = cv2.morphologyEx(mask_inv, cv2.MORPH_OPEN, kernel)
    
    # appy mask on original image
    segmented = cv2.bitwise_and(image, image, mask=mask_inv)
    
    if write_file:
        # write segemented image
        cv2.imwrite(path_out, segmented)
    
    return(mask_inv)

# set paths
file_name_in = 'Test7_1_warped.jpg'
file_name_out = 'Test7_1_warped_segmented.jpg'
path_in = '/mnt/gsdata/users/jaksztat/Work/Projects/DL_for_LAI_from_UAV_LiDAR/05_Validation_Data/Leaf_Area_Measurement/' + file_name_in
path_out = '/mnt/gsdata/users/jaksztat/Work/Projects/DL_for_LAI_from_UAV_LiDAR/05_Validation_Data/Leaf_Area_Measurement/' + file_name_out


# call segmentation
test = segment_objects(path_in)

In [7]:
np.unique_counts(test)

UniqueCountsResult(values=array([  0, 255], dtype=uint8), counts=array([  6392, 494598]))

In [30]:
# image dimension appr. 208mm x 295mm
measured = np.sum(test == 255) * (208*295)/(test.shape[0] * test.shape[1])
measured

np.float64(11784.508884360088)

In [31]:
# check (4 circels with 40mm diameter, 3 rectangles 40mm x 60mm)
real = 4* np.pi * 400 + 3 * 40 * 60
real

12226.54824574367

In [29]:
# relative error [%]
(measured-real)/real * 100

np.float64(-3.6154060205623924)